# Agentic RAG (Retrieval-Augmented Generation) system with LangChain and LangGraph.

### 1. Setup and Imports
First, install the necessary libraries. This example assumes you have an API key for a model like OpenAI or Anthropic (LangChain supports many).

In [ ]:
%pip install -qU langchain langchain-openai langgraph langchain-community faiss-cpu
%pip install pydantic==2.12.3


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage

# OLD/Incorrect Import (Causes the error)
#from langchain.vectorstores import FAISS
# NEW/Correct Import
from langchain_community.vectorstores import FAISS

from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator

In [ ]:
# --- 1. Set up Environment and LLM (Replace with your actual key and model) ---
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

# OpenAI API Key.

# For Google Colab environment.
from google.colab import userdata
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# For local environment.
#import os
#
#key = os.getenv("OPENAI_API_KEY")

if not key:
    raise ValueError("API key not found. Please set the MY_API_KEY environment variable.")

print("API Key loaded successfully!")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. Define the Agent State (LangGraph)
The agent's state dictates what information is passed between different steps (nodes) of the workflow.

In [ ]:
class AgentState(TypedDict):
    """Represents the state of our graph."""
    query: str  # The original user question
    context: Annotated[List[str], operator.add]  # List of retrieved document content
    answer: str # The final generated answer
    # A simple flag to control routing: True if documents were retrieved
    needs_rag: bool

### 3. Define the Retrieval Tool
This function sets up a simple in-memory RAG retriever (using FAISS) and wraps it as a tool the agent can use.

In [ ]:
# --- Mock Data and Retriever Setup ---
docs = [
    Document(page_content="LangChain is a framework for developing applications powered by language models."),
    Document(page_content="RAG stands for Retrieval-Augmented Generation, combining retrieval with LLMs."),
    Document(page_content="LangGraph allows creating stateful, multi-step agentic workflows."),
    Document(page_content="Agentic systems involve an LLM reasoning and deciding which tools to use."),
]

vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

@tool
def retrieve_docs(query: str) -> List[str]:
    """Search the internal vector store for documents relevant to the query."""
    retrieved_docs = retriever.invoke(query)
    return [doc.page_content for doc in retrieved_docs]

# The agent will only have access to this tool
tools = [retrieve_docs]

### 4. Define Graph Nodes (Functions)
These functions represent the steps (nodes) in the agent's workflow.

In [ ]:
def check_knowledge(state: AgentState) -> str:
    """Node 1: Decide if RAG is needed by asking the LLM."""
    query = state["query"]

    # Simple check prompt
    prompt = f"""
    You are a router agent. Your task is to determine if the following user query
    requires searching the internal knowledge base (RAG) or if you can answer it
    directly using your general knowledge.

    Query: "{query}"

    Respond with ONLY 'RAG' if the question is likely about LangChain, RAG,
    LangGraph, or Agentic systems. Otherwise, respond with ONLY 'DIRECT'.
    """

    # LLM call to route the query
    response = llm.invoke(prompt)
    decision = response.content.strip().upper()

    print(f"--- ROUTER DECISION: {decision} ---")

    # Update the state for the next conditional edge
    if decision == 'RAG':
        return "call_retriever"
    else:
        return "generate_direct_answer"

def call_retriever_node(state: AgentState) -> AgentState:
    """Node 2: Call the retrieve_docs tool."""
    print("--- CALLING RETRIEVER ---")
    query = state["query"]
    # Manually calling the tool for this simple agent
    retrieved_context = retrieve_docs.invoke(query)

    return {"context": retrieved_context, "query": query}

def generate_answer(state: AgentState) -> AgentState:
    """Node 3: Generate the final answer using context (if available)."""
    print("--- GENERATING FINAL ANSWER ---")
    query = state["query"]
    context = "\n".join(state["context"]) if state["context"] else "No specific context retrieved."

    final_prompt = f"""
    You are an expert Q&A system. Answer the user's question based ONLY
    on the provided context, if it is relevant. If the context is not
    relevant or the question is about general knowledge, answer based on
    your own knowledge.

    Context:
    ---
    {context}
    ---

    Question: {query}
    """

    response = llm.invoke(final_prompt)

    return {"answer": response.content, "query": query}

### 5. Build and Run the LangGraph
The graph defines the flow and decision points for the agent.

In [ ]:
# --- Build the Graph ---
workflow = StateGraph(AgentState)

# Define the nodes
workflow.add_node("router", check_knowledge)
workflow.add_node("retrieve", call_retriever_node)
workflow.add_node("generate", generate_answer)

# Set the start node
workflow.set_entry_point("router")

# Conditional edge from router
# If 'RAG' is returned, go to 'retrieve'; otherwise, go to 'generate'
workflow.add_conditional_edges(
    "router",
    lambda state: state["needs_rag"], # This is a placeholder, a real implementation would use the check_knowledge decision
    {
        "call_retriever": "retrieve",
        "generate_direct_answer": "generate"
    }
)

# RAG path: Retrieve -> Generate -> END
workflow.add_edge("retrieve", "generate")

# Both paths end at the final answer generation
workflow.add_edge("generate", END)

# Compile the graph
# NOTE: The conditional edge logic needs refinement to correctly use the
# decision from `check_knowledge` as an explicit conditional branch.
# For simplicity, we manually run the flow below to demonstrate the components.

# --- Example Run (Simplified Chain Execution) ---
def run_agentic_rag(query: str):
    # For a simple script, we can run the steps manually based on the router's output
    initial_state = {"query": query, "context": [], "answer": "", "needs_rag": False}

    # Step 1: Route
    decision = check_knowledge(initial_state)

    if decision == "call_retriever":
        # Step 2: Retrieve
        retrieved_state = call_retriever_node(initial_state)
        # Step 3: Generate
        final_state = generate_answer(retrieved_state)
    else:
        # Step 2: Generate directly
        final_state = generate_answer(initial_state)

    return final_state["answer"]

# --- Example Queries ---
query_rag = "What is RAG?"
query_direct = "What is the capital of France?"

print(f"\nQUERY 1: {query_rag}")
result_rag = run_agentic_rag(query_rag)
print(f"\nAGENT RESPONSE: {result_rag}")

print(f"\nQUERY 2: {query_direct}")
result_direct = run_agentic_rag(query_direct)
print(f"\nAGENT RESPONSE: {result_direct}")